<a href="https://colab.research.google.com/github/MatsuTaku/afkjourney-techcard-optimizer/blob/develop/%5BAFKJ%5D%E9%A0%98%E5%9C%B0_%E6%8A%80%E8%A1%93%E3%82%AB%E3%83%BC%E3%83%89%E3%82%B7%E3%83%9F%E3%83%A5%E3%83%AC%E3%83%BC%E3%82%BF%E3%83%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#技術カードシミュレータv0.03.0

カードの組み合わせによる生産アイテムの星数の出現確率を，動的計画法を用いて厳密に計算します

計算済みの結果は[こちら(Notion)](https://www.notion.so/AFK-Journey-24ce0075498480048a45fbc3b1ecacf4?source=copy_link)にある程度まとめてあります

下のフォームを確認し，上部バーの「すべてのセルを実行」を押してください．ページ最下部に結果が表示されます

*推奨：「ドライブにコピー」して利用することで，自身のGoogleDriveに各種変更が保存されるようになります．ただし，最新版を利用する際は[元のページ](https://colab.research.google.com/github/MatsuTaku/afkjourney-techcard-optimizer/blob/main/%5BAFKJ%5D%E9%A0%98%E5%9C%B0_%E6%8A%80%E8%A1%93%E3%82%AB%E3%83%BC%E3%83%89%E3%82%B7%E3%83%9F%E3%83%A5%E3%83%AC%E3%83%BC%E3%82%BF%E3%83%BC.ipynb)から再度「ドライブにコピー」してください*

[GitHub](https://github.com/MatsuTaku/afkjourney-techcard-optimizer)

In [ ]:
#@title Definitions
# !pip install numpy --quiet
!pip install tqdm --quiet

from collections import defaultdict
import numpy as np

items_kitchen = (
    'CircleBread',
    'PanCake',
    'Toast',
    'SpicyBread',
    'WineVinegar',
    'Dolma',
    'SaltyDryFruits',
    'SweetAndSpicyDryFruits',
    'ShroomStri-Fry',
    'UnknownItem(Kitchen)',
)
items_kitchen_jp = (
    '丸パン',
    'パンケーキ',
    'トースト',
    '旨辛パン',
    'ブドウ酢',
    'ブドウの塩漬け',
    '塩漬けドライフルーツ',
    '甘辛ドライフルーツ',
    'スパイシーキノコ炒め',
    '未知アイテム(厨房)',
)
items_workshop = (
    'IronCarvingKnife',
    'IromMask',
    'ThinCarvingKnife',
    'WoodenArrow',
    'WoodenBox',
    'CopperPot',
    'WoodenArmor',
    'ChillguardArmor',
    'UnknownItem(Forge)',
)
items_workshop_jp = (
    '鋼の彫刻刀',
    '鋼のマスク',
    '超薄刃彫刻刀',
    '木の矢',
    '木彫りの箱',
    '保温の銅鍋',
    '保温の木の鎧',
    '岩の鎧',
    '未知アイテム(鍛造工房)',
)
items_alchemy = (
    'GrowthPotion',
    'Bouquet',
    'Warmdust',
    'CalmingWarmdust',
    'UnknownItem(Alchemy)',
)
items_alchemy_jp = (
    '成長剤',
    '安らぎの花束',
    '温もり粉',
    '安らぎの温もり粉',
    '未知アイテム(錬金工房)',
)
levels_for_luminight = {
    'kitchen': {
        1: [0,1,1,0],
        2: [0,1,1,0],
        3: [0,2,1,0],
        4: [1,3,1,0],
        5: [1,4,1,1],
        6: [2,4,1,2],
        7: [3,5,1,2],
        8: [4,5,1,2,1],
        9: [4,5,1,3,1],
        10: [4,6,1,3,1],
        11: [4,6,1,3,1],
    },
    'workshop': {#['熟練鍛造', '鍛造', '点火', '加熱', 'チャージ']
        1: [0,1,1,0,0],
        2: [0,1,1,0,0],
        3: [0,2,1,0,0],
        4: [0,3,1,1,0],
        5: [1,3,1,1,0],
        6: [2,4,1,1,1],
        7: [3,4,1,2,1],
        8: [3,4,1,2,2],
        9: [3,4,1,2,2],
        10: [3,4,1,2,2],
        11: [3,5,1,2,2],
    },
    'alchemy': {#['材料投入', '研磨', '蒸留', '魔力注入']
        1: [1,1,0,0],
        2: [1,1,0,0],
        3: [1,1,1,0],
        4: [1,2,1,1],
        5: [2,2,1,1],
        6: [2,3,1,1],
        7: [3,3,1,1],
        8: [3,4,1,2],
        9: [4,4,1,2],
        10: [4,4,1,2],
        11: [4,4,1,2],
    }
}
card_limits_map = {
    'Kitchen': {# ['火加減把握Lv.1', '切り分けLv.4', '調味Lv.1', '弱火煮込みLv.1']
        1: [0,2,1,0],
        2: [0,2,1,0],
        3: [3,3,1,0],
        4: [3,3,1,1],
        5: [4,3,1,2],
        6: [4,3,1,2],
        7: [4,3,1,2,2],
        8: [4,3,1,2,2],#temporary
        9: [4,3,1,2,2],
        10: [4,3,1,2,2],
        },
    'Workshop': {#['熟練鍛造', '鍛造', '点火', '加熱', 'チャージ']
        1: [0,2,1,0,0],
        2: [0,2,1,0,0],
        3: [0,2,1,1,0],
        4: [2,3,1,1,0],
        5: [3,3,1,1,1],
        6: [4,3,1,2,2],
        7: [4,3,1,2,2],
        8: [4,3,1,2,2],#temporary
        9: [4,3,1,2,2],
        10: [4,3,1,2,2],
        },
    'Alchemy': {#['材料投入', '研磨', '蒸留', '魔力注入']
        1: [2,2,0,0],
        2: [2,2,1,0],
        3: [2,2,1,0],
        4: [2,2,1,2],
        5: [2,2,1,2],
        6: [3,3,1,2],
        7: [4,4,1,2],
        8: [4,4,1,2],#temporary
        9: [4,4,1,2],
        10: [4,4,1,2],
        },
}
build_item_all = items_kitchen + items_workshop + items_alchemy
build_item_jp_all = items_kitchen_jp + items_workshop_jp + items_alchemy_jp
print(build_item_jp_all)
build_item_jp = '温もり粉' #@param ['丸パン', 'パンケーキ', 'トースト', '旨辛パン', 'ブドウ酢', 'ブドウの塩漬け', '塩漬けドライフルーツ', '甘辛ドライフルーツ', 'スパイシーキノコ炒め', '鋼の彫刻刀', '鋼のマスク', '超薄刃彫刻刀', '木の矢', '木彫りの箱', '保温の銅鍋', '保温の木の鎧', '岩の鎧', '成長剤', '安らぎの花束', '温もり粉', '安らぎの温もり粉', '未知アイテム(厨房)', '未知アイテム(鍛造工房)', '未知アイテム(錬金工房)']
build_item = build_item_all[build_item_jp_all.index(build_item_jp)]
#@markdown ★ランク条件(未知アイテム用)
star_rank_threshold_for_unsopported_item = 1000 #@param {type:"number"}
#@markdown カード枚数(未知アイテム用)
cound_cards_for_unsopported_item = 7 #@param {type:"number"}
COUNT_CARDS = {
    # Kitchen
    'CircleBread': 3,
    'PanCake': 4,
    "Toast": 5,
    'SpicyBread': 5,
    'WineVinegar': 6,
    'Dolma': 4,
    'SweetAndSpicyDryFruits': 7,
    'SaltyDryFruits': 5,
    'ShroomStri-Fry': 6,
    'UnknownItem(Kitchen)': cound_cards_for_unsopported_item,
    # ForgingWorkshop
    'IronCarvingKnife': 3,
    'IromMask': 4,
    'ThinCarvingKnife': 5,
    'WoodenArrow': 5,
    'WoodenBox': 4,
    'CopperPot': 5,
    'WoodenArmor': 6,
    'ChillguardArmor': 7,
    'UnknownItem(Forge)': cound_cards_for_unsopported_item,
    # AlchemyWorkshop
    'GrowthPotion': 3,
    'Bouquet': 6,
    'Warmdust': 7,
    'CalmingWarmdust': 7,
    'UnknownItem(Alchemy)': cound_cards_for_unsopported_item,
}
STAR_NEEDS = {
    # Kitchen
    'CircleBread': [0,6,8],
    'PanCake': [0,15,25],
    "Toast": [0,102,159],
    'SpicyBread': [0,372,448,635],
    'WineVinegar': [0,1070,1270,1420,1740],
    'Dolma': [0,344,440,462,647],
    'SweetAndSpicyDryFruits': [0,2670,2950,3340,4020],
    'SaltyDryFruits': [0,1040,1230,1350,1640],
    'ShroomStri-Fry': [0,2160,2450,2810,3520],
    'UnknownItem(Kitchen)': [0,star_rank_threshold_for_unsopported_item],
    # ForgingWorkshop
    'IronCarvingKnife': [0,6,8],
    'IromMask': [0,19,23],
    'ThinCarvingKnife': [0, 46, 86],
    'WoodenArrow': [0, 108, 130, 163],
    'WoodenBox': [0, 152, 180, 182, 190],
    'CopperPot': [0, 566, 675, 797, 1040],
    'WoodenArmor': [0,762,873,974,1190],
    'ChillguardArmor': [0,2210,2600,2920,3770],
    'UnknownItem(Forge)': [0,star_rank_threshold_for_unsopported_item],
    # AlchemyWorkshop
    'GrowthPotion': [0,12,27],
    'Bouquet': [0,518,575,637,744],
    'Warmdust': [0,1510,1630,1760,2040],
    'CalmingWarmdust': [0,2150,2350,2520,2860],
    'UnknownItem(Alchemy)': [0,star_rank_threshold_for_unsopported_item],
}

#@markdown 厨房
kitchen_level = 7 #@param {type:"number", label:"厨房"}
#@markdown - 火加減把握
heat_controll_card_level = 4 #@param {type:"number", label:"火加減把握"}
#@markdown - 切り分け
cut_card_level = 4 #@param {type:"number", label:"切り分け"}
cook_card_level = 1
#@markdown - 弱火煮込み
low_heat_card_level = 2 #@param {type:"number", label:"弱火煮込み"}
#@markdown - 発酵
ferment_card_level = 1 #@param {type:"number", label:"発酵"}

#@markdown 鍛造工房
forge_level = 7 #@param {type:"number", label:"鍛造工房"}
#@markdown - 熟練鍛造
skilled_forging_level = 2 #@param {type:"number", label:"熟練鍛造"}
#@markdown - 鍛造
forging_level = 4 #@param {type:"number", label:"鍛造"}
ignition_level = 1
#@markdown - 加熱
burning_level = 1 #@param {type:"number", label:"加熱"}
#@markdown - チャージ
charge_level = 2 #@param {type:"number", label:"チャージ"}

#@markdown 錬金工房
alchemy_level = 7 #@param {type:"number", label:"錬金工房"}
#@markdown - 材料投入
material_input_level = 3 #@param {type:"number", label:"材料投入"}
#@markdown - 研磨
grind_level = 4 #@param {type:"number", label:"研磨"}
distillation_level = 1 # 蒸留
#@markdown - 魔力注入
magic_infusion_level = 1 #@param {type:"number", label:"魔力注入"}

In [ ]:
# @title Algorithms
DP_TABLE_SIZE = 100
DP_SHAPE = (DP_TABLE_SIZE, DP_TABLE_SIZE)
ADD_TYPE_RANDOM = "ADD_TYPE_RANDOM"
ADD_TYPE_ALL = "ADD_TYPE_ALL"
ADD_TYPE_MAX = "ADD_TYPE_MAX"
ADD_TYPE_MIN = "ADD_TYPE_MIN"

def dp_add_shift(dp, shift_shape):
    ret = np.pad(dp, ((shift_shape[0],0), (shift_shape[1],0)), mode='constant', constant_values=0)
    sum_outbounds = np.sum(ret[DP_TABLE_SIZE:,:]) + np.sum(ret[:,DP_TABLE_SIZE:]) - np.sum(ret[DP_TABLE_SIZE:,DP_TABLE_SIZE:])
    ret[DP_TABLE_SIZE-1,DP_TABLE_SIZE-1] += sum_outbounds
    return ret[:DP_TABLE_SIZE, :DP_TABLE_SIZE]

def dp_minus_shift(dp, shift_shape):
    ret = np.pad(dp, ((0,shift_shape[0]), (0,shift_shape[1])), mode='constant', constant_values=0)
    ret[-DP_TABLE_SIZE,-DP_TABLE_SIZE] += np.sum(ret[:-DP_TABLE_SIZE,:-DP_TABLE_SIZE])
    ret[-DP_TABLE_SIZE, -DP_TABLE_SIZE:] += np.sum(ret[:-DP_TABLE_SIZE, -DP_TABLE_SIZE:], axis=0)
    ret[-DP_TABLE_SIZE:, -DP_TABLE_SIZE] += np.sum(ret[-DP_TABLE_SIZE:, :-DP_TABLE_SIZE], axis=1)
    return ret[-DP_TABLE_SIZE:, -DP_TABLE_SIZE:]

def random_add(dp, add_unit):
    xadd = dp_add_shift(dp, (add_unit,0))
    yadd = dp_add_shift(dp, (0, add_unit))
    return (xadd+yadd)/2

def all_add(dp, add_unit):
    return dp_add_shift(dp, (add_unit,add_unit))

def max_add(dp, add_unit):
    ilj = np.zeros_like(dp)
    iej = np.zeros_like(dp)
    igj = np.zeros_like(dp)
    for i in range(DP_TABLE_SIZE):
        for j in range(DP_TABLE_SIZE):
            if i<j:
                ilj[i,j] = dp[i,j]
            elif i>j:
                igj[i,j] = dp[i,j]
            else:
                iej[i,j] = dp[i,j]
    ilj = dp_add_shift(ilj, (0,add_unit))
    igj = dp_add_shift(igj, (add_unit,0))
    iej = random_add(iej, add_unit)
    return ilj+iej+igj

def min_add(dp, add_unit):
    ilj = np.zeros_like(dp)
    iej = np.zeros_like(dp)
    igj = np.zeros_like(dp)
    for i in range(DP_TABLE_SIZE):
        for j in range(DP_TABLE_SIZE):
            if i<j:
                ilj[i,j] = dp[i,j]
            elif i>j:
                igj[i,j] = dp[i,j]
            else:
                iej[i,j] = dp[i,j]
    ilj = dp_add_shift(ilj, (add_unit,0))
    igj = dp_add_shift(igj, (0,add_unit))
    iej = random_add(iej, add_unit)
    return ilj+iej+igj

def max_double(dp):
    ret = np.zeros_like(dp)
    for i in range(DP_TABLE_SIZE):
        for j in range(DP_TABLE_SIZE):
            if i<j:
                nj=min(j*2,DP_TABLE_SIZE-1)
                ret[i,nj] += dp[i,j]
            elif i>j:
                ni=min(i*2,DP_TABLE_SIZE-1)
                ret[ni,j] += dp[i,j]
            else:
                ni=min(i*2,DP_TABLE_SIZE-1)
                nj=min(j*2,DP_TABLE_SIZE-1)
                ret[ni,j] += dp[i,j]/2
                ret[i,nj] += dp[i,j]/2
    return ret

def random_minus(dp, minus_unit):
    xminus = dp_minus_shift(dp, (minus_unit,0))
    yminus = dp_minus_shift(dp, (0, minus_unit))
    return (xminus+yminus)/2

def all_minus(dp, minus_unit):
    return dp_minus_shift(dp, (minus_unit,minus_unit))

def min_add_max_minus(dp, add_unit, minus_unit):
    ilj = np.zeros_like(dp)
    iej = np.zeros_like(dp)
    igj = np.zeros_like(dp)
    for i in range(DP_TABLE_SIZE):
        for j in range(DP_TABLE_SIZE):
            if i<j:
                ilj[i,j] = dp[i,j]
            elif i>j:
                igj[i,j] = dp[i,j]
            else:
                iej[i,j] = dp[i,j]
    ilj = dp_add_shift(ilj, (0,add_unit))
    ilj = dp_minus_shift(ilj, (minus_unit, 0))
    igj = dp_add_shift(igj, (add_unit,0))
    igj = dp_minus_shift(igj, (0, minus_unit))
    iejx = dp_add_shift(iej, (0,add_unit))
    iejx = dp_minus_shift(iejx, (minus_unit, 0))
    iejy = dp_add_shift(iej, (add_unit,0))
    iejy = dp_minus_shift(iejy, (0, minus_unit))
    return ilj+igj+(iejx+iejy)/2

def max_add_min_minus(dp, add_unit, minus_unit):
    ilj = np.zeros_like(dp)
    iej = np.zeros_like(dp)
    igj = np.zeros_like(dp)
    for i in range(DP_TABLE_SIZE):
        for j in range(DP_TABLE_SIZE):
            if i<j:
                ilj[i,j] = dp[i,j]
            elif i>j:
                igj[i,j] = dp[i,j]
            else:
                iej[i,j] = dp[i,j]
    ilj = dp_add_shift(ilj, (add_unit, 0))
    ilj = dp_minus_shift(ilj, (0, minus_unit))
    igj = dp_add_shift(igj, (0, add_unit))
    igj = dp_minus_shift(igj, (minus_unit, 0))
    iejx = dp_add_shift(iej, (0,add_unit))
    iejx = dp_minus_shift(iejx, (minus_unit, 0))
    iejy = dp_add_shift(iej, (add_unit,0))
    iejy = dp_minus_shift(iejy, (0, minus_unit))
    return ilj+igj+(iejx+iejy)/2

In [ ]:
# @title Kitchen Card Classes
class Card:
    def __init__(self, level, name, jname):
        self.level = level
        self.name = name
        self.jname = jname

    def update_dp(self, dp_old, trans_prop, stack_table, idx, n):
        return dp_old.copy()

HEAT_CONTROLL_ID = 0
CUT_ID = 1
COOK_ID = 2
LOW_HEAT_ID = 3
FERMENT_ID = 4

class CutCard(Card):
    ADD_UNIT_MIN = [0,1,2,3,4,8]
    ADD_UNIT_MAX = [0,1,2,3,8,12]
    def __init__(self, level):
        super().__init__(level, 'CutCard', '切り分け')

    def update_dp(self, dp_old, trans_prop, stack_table, idx, n):
        pdp = dp_old * trans_prop
        lo = CutCard.ADD_UNIT_MIN[self.level]
        hi = CutCard.ADD_UNIT_MAX[self.level]
        if build_item == 'Dolma':
            lo = hi
        ret = np.zeros_like(dp_old)
        for d in range(lo, hi+1):
            dx = dp_add_shift(pdp, (d,0))
            dy = dp_add_shift(pdp, (0,d))
            ret += dx
            ret += dy
        return ret / (2*(hi+1-lo))

class CookCard(Card):
    def __init__(self, level):
        super().__init__(level, 'CookCard', '調味')

    def update_dp(self, dp_old, trans_prop, stack_table, idx, n):
        pdp = dp_old * trans_prop
        mx = [2,1]
        my = [1,2]
        ret = np.zeros_like(dp_old)
        for i in range(DP_TABLE_SIZE):
            for j in range(DP_TABLE_SIZE):
                for k in range(2):
                    ni = i*mx[k]
                    nj = j*my[k]
                    ni = min(ni,DP_TABLE_SIZE-1)
                    nj = min(nj,DP_TABLE_SIZE-1)
                    ret[ni,nj] += pdp[i,j]
        return ret/2

CMB_LIM = 10
fact = [1 for _ in range(CMB_LIM)]
for i in range(1,CMB_LIM):
    fact[i] = fact[i-1]*i

def cmb(n,r):
    if n<r:
        return 0
    return fact[n]/(fact[r]*fact[n-r])

class LowHeat(Card):
    ADD_UNIT = [0,4,2,4]
    ADD_TYPE = ["-", ADD_TYPE_RANDOM, ADD_TYPE_ALL, ADD_TYPE_ALL]
    def __init__(self, level):
        self.add_unit = LowHeat.ADD_UNIT[level]
        self.add_type = LowHeat.ADD_TYPE[level]
        super().__init__(level, 'LowHeat', '弱火煮込み')

    def hooked_dp(self, dp_old, trans_prop):
        pdp = dp_old * trans_prop
        if self.add_type == ADD_TYPE_RANDOM:
            return random_add(pdp, self.add_unit)
        elif self.add_type == ADD_TYPE_ALL:
            return all_add(pdp, self.add_unit)

class HeatControllCard(Card):
    ADD_UNIT = [0,3,4,6,8]
    def __init__(self, level):
        self.add_unit = HeatControllCard.ADD_UNIT[level]
        super().__init__(level, 'HeatControllCard', '火加減把握')
        self.continue_prop = 0.5

    def update_dp(self, dp_old, trans_prop, stack_table, idx, n):
        dp = [np.zeros((DP_TABLE_SIZE, DP_TABLE_SIZE)) for _ in range(20)]
        dp[0] = dp_old * trans_prop
        low_heat = LowHeat(low_heat_card_level)
        for k in range(1,20):
            p_cont = self.continue_prop if k>1+(Ferment(ferment_card_level).force_count if stack_table[FERMENT_ID]>0 else 0) else 1
            pdp = dp[k-1] * p_cont
            dp[k][:] = random_add(pdp, self.add_unit)
            for low_heats in range(stack_table[LOW_HEAT_ID]):
                dp[k][:] = low_heat.hooked_dp(dp[k], 1)
            if k>1:
                dp[k-1] *= (1-p_cont)
        return np.sum(dp[1:], axis=0)

class Ferment(Card):
    FORCE_COUNT = [0,1,2]
    def __init__(self, level):
        super().__init__(level, 'Ferment', '発酵')
        self.force_count = Ferment.FORCE_COUNT[level]

In [ ]:
# @title Forging Workshop Card Classes

SKILLED_FORGING_ID = 0
FORGING_ID = 1
IGNITION_ID = 2
BURNING_ID = 3
CHARGE_ID = 4

class Burning(Card):
    BONUS_UNIT = [0,3,6]
    def __init__(self, level):
        self.bonus_unit = Burning.BONUS_UNIT[level]
        super().__init__(level, 'Burning', '加熱')
class Charge(Card):
    CHARGE_COUNT = [0,1,3]
    def __init__(self, level):
        super().__init__(level, 'Charge', 'チャージ')
        self.charge_count = Charge.CHARGE_COUNT[level]
COPPER_POT_BONUS = 0.3
class SkilledForging(Card):
    ADD_UNIT = [0,3,4,4]
    BONUS_UNIT = [0,3,3,4]
    def __init__(self, level):
        self.add_unit = SkilledForging.ADD_UNIT[level]
        self.bonus_unit = SkilledForging.BONUS_UNIT[level]
        super().__init__(level, 'SkilledForging', '熟練鍛造')
    def update_dp(self, dp_old, trans_prop, stack_table, idx, n):
        pdp = dp_old * trans_prop
        add_unit = self.add_unit
        add_unit += self.bonus_unit * stack_table[SKILLED_FORGING_ID]
        add_unit += Burning(burning_level).bonus_unit * stack_table['Burning']
        charge = stack_table[CHARGE_ID]>0
        if charge:
            stack_table[CHARGE_ID] -= 1
        f = all_add if charge else random_add
        ret = f(pdp, add_unit)
        if build_item == 'CopperPot':
            ret = ret * (1-COPPER_POT_BONUS) + f(ret, add_unit) * COPPER_POT_BONUS
        return ret
class Forging(Card):
    ADD_UNIT = [0,1,2,3,5,10]
    def __init__(self, level):
        self.add_unit = Forging.ADD_UNIT[level]
        super().__init__(level, 'Forging', '鍛造')
    def update_dp(self, dp_old, trans_prop, stack_table, idx, n):
        pdp = dp_old * trans_prop
        add_unit = self.add_unit
        add_unit += Burning(burning_level).bonus_unit * stack_table[BURNING_ID]
        charge = stack_table[CHARGE_ID]>0
        if charge:
            stack_table[CHARGE_ID] -= 1
        if charge or (build_item=='WoodenBox' and stack_table[FORGING_ID]==0):
            return all_add(pdp, add_unit)
        else:
            return random_add(pdp, add_unit)
class Ignition(Card):
    ADD_UNIT = [0,2]
    def __init__(self, level):
        self.add_unit = Ignition.ADD_UNIT[level]
        super().__init__(level, 'Ignition', '点火')
    def update_dp(self, dp_old, trans_prop, stack_table, idx, n):
        pdp = dp_old * trans_prop
        return random_add(pdp, self.add_unit)

In [ ]:
# @title Alchemy Workshop Card Classes
MATERIAL_INPUT_ID = 0
GRIND_ID = 1
DISTILL_ID = 2
MASIC_INFUSION_ID = 3

class MaterialInput(Card):
    ADD_UNIT = [0,4,6,10]
    MINUS_UNIT = [0,0,0,2]
    def __init__(self, level):
        super().__init__(level, 'MaterialInput', '材料投入')
        self.add_type = ADD_TYPE_MAX
        self.add_unit = MaterialInput.ADD_UNIT[level]
        self.minus_unit = MaterialInput.MINUS_UNIT[level]
    def update_dp(self, dp_old, trans_prop, stack_table, idx, n):
        pdp = dp_old * trans_prop
        pdp = MagicInfusion(magic_infusion_level).hooked_dp(pdp, stack_table)
        ret = max_add_min_minus(pdp, self.add_unit, self.minus_unit)
        if build_item == 'Warmdust':
            ret = min_add(ret, 1)
        if build_item == 'CalmingWarmdust':
            ret = min_add(ret, 3)
        return ret

class Grind(Card):
    ADD_UNIT = [0,2,3,4,6]
    MINUS_UNIT = [0,0,0,2,3]
    def __init__(self, level):
        super().__init__(level, 'Grind', '研磨')
        self.add_unit = Grind.ADD_UNIT[level]
        self.minus_unit = Grind.MINUS_UNIT[level]
    def update_dp(self, dp_old, trans_prop, stack_table, idx, n):
        pdp = dp_old * trans_prop
        pdp = MagicInfusion(magic_infusion_level).hooked_dp(pdp, stack_table)
        ret = min_add_max_minus(pdp, self.add_unit, self.minus_unit)
        if build_item == 'Warmdust':
            ret = min_add(ret, 1)
        if build_item == 'CalmingWarmdust':
            ret = min_add(ret, 3)
        return ret

class Distill(Card):
    def __init__(self, level):
        super().__init__(level, 'Distill', '蒸留')
    def update_dp(self, dp_old, trans_prop, stack_table, idx, n):
        pdp = dp_old * trans_prop
        pdp = MagicInfusion(magic_infusion_level).hooked_dp(pdp, stack_table)
        ret = max_double(pdp)
        if build_item == 'Warmdust':
            ret = min_add(ret, 1)
        if build_item == 'CalmingWarmdust':
            ret = min_add(ret, 3)
        return ret

class MagicInfusion(Card):
    ADD_UNIT = [0,8,16]
    BONUS_MINUS_UNIT = [0,1,1]
    BONUS_MINUS_TYPE = ["-", ADD_TYPE_RANDOM, ADD_TYPE_ALL]
    def __init__(self, level):
        super().__init__(level, 'MagicInfusion', '魔力注入')
        self.add_unit = MagicInfusion.ADD_UNIT[level]
        self.bonus_minus_unit = MagicInfusion.BONUS_MINUS_UNIT[level]
        self.bonus_minus_type = MagicInfusion.BONUS_MINUS_TYPE[level]
    def update_dp(self, dp_old, trans_prop, stack_table, idx, n):
        pdp = dp_old * trans_prop
        pdp = MagicInfusion(magic_infusion_level).hooked_dp(pdp, stack_table)
        ret = min_add(pdp, self.add_unit)
        if build_item == 'Warmdust':
            ret = min_add(ret, 1)
        if build_item == 'CalmingWarmdust':
            ret = min_add(ret, 3)
        return ret
    def hooked_dp(self, dp_old, stack_table):
        ret = dp_old*1
        f = random_minus if self.bonus_minus_type == ADD_TYPE_RANDOM else all_minus if self.bonus_minus_type == ADD_TYPE_ALL else None
        for i in range(stack_table[MASIC_INFUSION_ID]):
            ret = f(ret, self.bonus_minus_unit)
        return ret

In [ ]:
# @title Calculation Prepare
card_classes_kitchen = [
    HeatControllCard(heat_controll_card_level),
    CutCard(cut_card_level),
    CookCard(cook_card_level),
    LowHeat(low_heat_card_level)
]
card_classes_workshop = [
    SkilledForging(skilled_forging_level),
    Forging(forging_level),
    Ignition(ignition_level),
    Burning(burning_level),
    Charge(charge_level),
]
card_classes_alchemy = [
    MaterialInput(material_input_level),
    Grind(grind_level),
    Distill(distillation_level),
    MagicInfusion(magic_infusion_level),
]
card_classes_map = {
    'Kitchen': card_classes_kitchen,
    'Workshop': card_classes_workshop,
    'Alchemy': card_classes_alchemy,
}

In [ ]:
# @title Calculation
count_cards = COUNT_CARDS[build_item]
workplace = 'Kitchen' if build_item in items_kitchen else 'Workshop' if build_item in items_workshop else 'Alchemy' if build_item in items_alchemy else None
workplace_jp = {
    'Kitchen': '厨房',
    'Workshop': '鍛造工房',
    'Alchemy': '錬金工房',
}
card_classes = card_classes_map[workplace]
workplace_level = {
    'Kitchen': kitchen_level,
    'Workshop': forge_level,
    'Alchemy': alchemy_level,
}
card_limits = card_limits_map[workplace][workplace_level[workplace]]

import itertools
def for_perm_list(caps):
    ord = []
    for i,c in enumerate(caps):
        ord += [i for _ in range(c)]
    for perm in itertools.permutations([i for i in range(count_cards)]):
        yield [ord[i] for i in perm]

card_set = [-1 for _ in range(len(card_limits))]
def for_card_set(d):
    if sum(card_set)>count_cards:
        return
    if d==len(card_set):
        if sum(card_set)==count_cards:
            yield card_set.copy()
        return
    for k in range(card_limits[d]+1):
        card_set[d] = k
        yield from for_card_set(d+1)
    card_set[d] = -1

star_needs = STAR_NEEDS[build_item]
def get_probs(tb):
    res = [0 for _ in range(len(star_needs))]
    for i in range(DP_TABLE_SIZE):
        for j in range(DP_TABLE_SIZE):
            eval = i*j
            v = tb[i,j]
            for c in range(len(star_needs)-1,-1,-1):
                if eval >= star_needs[c]:
                    res[c] += v
                    break
    # print((tb[:20,:20]*100).astype(int))
    # print(cs, '|', (np.round(np.array(res)*100)).astype(int))
    return res

def dp_forall_permutation(cmb):
    perm_list = list(tuple(p) for p in for_perm_list(cmb))
    perm_cnt_list = defaultdict(int)
    for p in perm_list:
        perm_cnt_list[tuple(p)] += 1
    cnt_perm = len(perm_list)
    # print(cmb, cnt_perm, len(perm_cnt_list))
    sdp = np.zeros((DP_TABLE_SIZE,DP_TABLE_SIZE))
    for perm,c in perm_cnt_list.items():
        dp = np.zeros((DP_TABLE_SIZE,DP_TABLE_SIZE))
        dp[1,1] = 1
        # card_status = CardStatus('Kitchen')
        card_stack = defaultdict(int)
        for ci in perm:
            ndp = card_classes[ci].update_dp(dp,1,card_stack,ci,len(perm))
            if not abs(np.sum(dp)-1)<1e-3:
                print(card_classes[ci].name, np.sum(ndp))
            assert(abs(np.sum(dp)-1)<1e-3)
            dp = ndp
            card_stack[card_classes[ci].name] += 1
            if card_classes[ci].name == 'Charge':
                card_stack[card_classes[ci].name] = card_classes[ci].charge_count
            # print(np.round(dp[:18,:18]*100))
        sdp += dp * c
    sdp /= cnt_perm
    return sdp

def dp_cardset(cmb):
    remain_table = cmb.copy()
    dp = defaultdict(lambda: defaultdict(lambda: np.zeros((DP_TABLE_SIZE,DP_TABLE_SIZE))))
    idpt = np.zeros((DP_TABLE_SIZE,DP_TABLE_SIZE))
    idpt[1,1] = 1
    dp[tuple(cmb)][tuple(0 for _ in range(len(cmb)))] = idpt
    for t in range(count_cards):
        pdp = defaultdict(lambda: defaultdict(lambda: np.zeros((DP_TABLE_SIZE,DP_TABLE_SIZE))))
        pdp,dp = dp,pdp
        nsum = 0
        for remain_table,ddp in pdp.items():
            # print("R", remain_table)
            for stack_table, dddp in ddp.items():
                # print("S", stack_table)
                for ci in range(len(cmb)):
                    # print(ci)
                    if remain_table[ci]==0:
                        continue
                    select_prop = remain_table[ci]/(count_cards-t)
                    cstack_table = list(stack_table)
                    cremain_table = list(remain_table)
                    ndpt = card_classes[ci].update_dp(dddp, select_prop, cstack_table, t, count_cards)
                    # print(np.sum(dddp), select_prop, np.sum(dddp)*select_prop, np.sum(ndpt))
                    assert(np.sum(dddp)*select_prop-np.sum(ndpt)<1e-6)
                    cstack_table[ci] += 1
                    cremain_table[ci] -= 1
                    # print(cremain_table, cstack_table, np.sum(ndpt))
                    dp[tuple(cremain_table)][tuple(cstack_table)] += ndpt
                    nsum += np.sum(ndpt)
        # print(t, nsum)
        assert(abs(nsum-1) < 1e-6)
    alldp = np.zeros((DP_TABLE_SIZE,DP_TABLE_SIZE))
    for ddp in pdp.values():
        for dddp in ddp.values():
            alldp += dddp
    return alldp

results = []
from tqdm.notebook import tqdm
DP_MODE_PERMUTATION = 'Dp with Permutation'
DP_MODE_CARDSET = 'Dp for cardset'
DP_MODE = 'Dp for cardset' #@param ['Dp with Permutation', 'Dp for cardset']
for cmb in tqdm(list(for_card_set(0))):
    sdp = None
    if DP_MODE == DP_MODE_PERMUTATION:
        sdp = dp_forall_permutation(cmb)
    elif DP_MODE == DP_MODE_CARDSET:
        sdp = dp_cardset(cmb)
    else:
        assert(False)

    # show_dp(tuple(cmb),sdp)
    # print(np.round(sdp[:18,:18]*100))
    results.append((tuple(cmb), get_probs(sdp)))

results.sort(key=lambda x: x[1])

print('生産アイテム:', build_item_jp, workplace_jp[workplace], 'Lv.', workplace_level[workplace], '★条件', star_needs)
print('カード一覧:', [c.jname+'Lv.'+str(c.level) for c in card_classes])
print('カード内訳 |', '★ランク確率[%]')
for cs, res in results:
    print(cs, '|', (np.round(np.array(res)*100)).astype(int))